# Make pseudobulks

For each cancer sample we will add it to a pseudobulk make of single-cell ATAC-Seq from blood.
To do the titration analysis, we will do pseudobulks of different number of blood cells used.


Percentage is calculated as the proportion of total counts per sample coming from each source


100 cells = 9% blood

500 cells = 30% blood 

1000 cells = 50% blood 

5000 cells = 80% blood 


In [1]:
import sys
sys.path.insert(1, '../../')
sys.path.insert(1, '../')
from buddi import buddi
from buddi.preprocessing import sc_preprocess

# general imports
import numpy as np
import os
import pandas as pd
import scanpy as sc
import anndata as ad


# Images, plots, display, and visualization
import pandas as pd

# matplotlib settings for Jupyter notebooks only
%matplotlib inline

import pickle
from pathlib import Path


2024-04-18 14:23:07.130727: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-18 14:23:07.131790: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-18 14:23:07.153606: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-18 14:23:07.154275: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-18 14:23:07.524191: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT

# Parameters

In [2]:
# parameters
data_path = f"{os.getcwd()}/../data/"

aug_data_path = f"{data_path}/pseudobulks/"



#####################
### set the study ###
#####################

res_name = "all-tcga"
tcga_file = f"{data_path}/TCGA/tcga_atac_promoters.h5ad"
pbmc_file = f"{data_path}/PBMC/pbmc10k_processed.h5ad"



In [3]:
# set paths



# get TCGA
tcga_adata = ad.read_h5ad(tcga_file)

# get blood PBMC
pbmc_adata = ad.read_h5ad(pbmc_file)

# rescale

Since we are combining single cell and atac, we need to make sure that we are on and expected scaling

In [4]:
# get the median sum for each single cell blood 
np.median([np.sum(pbmc_adata.X[idx]) for idx in np.random.choice(pbmc_adata.X.shape[0], 1000) ])

20694.5

In [5]:
# get the median sum for each tcga sample
np.median([np.sum(tcga_adata.X[idx]) for idx in range(tcga_adata.X.shape[0])])

5669622.0

In [6]:
# now we normalize the sums so that TCGA is 1000 times larger than single cell
sc.pp.normalize_total(tcga_adata, target_sum=1e7) 
sc.pp.normalize_total(pbmc_adata, target_sum=1e4)

# pseudobulks

We will do 4 experiments, 50, 100, 500, 1000 cells used to make blood pseudobulks.

In each of these experiments we will use 10 samples.

In all of those 10 samples, we will add one of the tcga samples.

21000 pseudobulks total: 4x10x525

In [7]:
tcga_adata

AnnData object with n_obs × n_vars = 525 × 45782
    obs: 'sample_id', 'scpred_CellType'
    var: 'seqnames', 'start', 'end', 'name', 'score', 'annotation'

In [8]:
tcga_adata.obs.scpred_CellType.value_counts()

BRCA      141
KIDNEY    100
COAD       81
LUNG       76
PRAD       52
STAD       41
LIHC       34
Name: scpred_CellType, dtype: int64

In [9]:
tcga_adata.obs.shape[0]

525

### train / test split

Split first the PBMCs, then TCGA

In [10]:
### now split test and train pbmc cells
# generate cell-type specific split
pbmc_adata.obs["isTraining"] = "Train"
pbmc_adata.obs["test_train_key"] = pbmc_adata.obs["scpred_CellType"].astype(str)

# for each element in the key
# split by 50%
for curr_key in pbmc_adata.obs["test_train_key"].unique():
  curr_idx = np.where(pbmc_adata.obs["test_train_key"] == curr_key)[0]
  num_sample = np.floor(len(curr_idx)/2).astype(int)
  test_idx = np.random.choice(curr_idx, num_sample)
  pbmc_adata.obs["isTraining"][test_idx] = "Test"
pd.set_option('display.max_rows', 160)


/tmp/ipykernel_1781024/2715567430.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pbmc_adata.obs["isTraining"][test_idx] = "Test"


In [11]:

tab = pbmc_adata.obs.groupby(['test_train_key', 'isTraining']).size()

tab.unstack()


isTraining,Test,Train
test_train_key,,
CD14 mono,713,1107
CD16 mono,183,273
CD4+ memory T,703,1057
CD4+ naïve T,624,986
CD8+ activated T,323,498
CD8+ naïve T,603,898
MAIT,49,76
NK,204,345
intermediate mono,417,638


In [12]:
### now split test and train TCGA samples

# generate cell-type specific split
tcga_adata.obs["isTraining"] = "Train"
tcga_adata.obs["test_train_key"] = tcga_adata.obs["scpred_CellType"].astype(str)

# for each element in the key
# split by 50%
for curr_key in tcga_adata.obs["test_train_key"].unique():
  curr_idx = np.where(tcga_adata.obs["test_train_key"] == curr_key)[0]
  num_sample = np.floor(len(curr_idx)/2).astype(int)
  test_idx = np.random.choice(curr_idx, num_sample)
  tcga_adata.obs["isTraining"][test_idx] = "Test"
pd.set_option('display.max_rows', 160)


/tmp/ipykernel_1781024/3382582093.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tcga_adata.obs["isTraining"][test_idx] = "Test"


In [13]:

tab = tcga_adata.obs.groupby(['test_train_key', 'isTraining']).size()

tab.unstack()


isTraining,Test,Train
test_train_key,,
BRCA,52,89
COAD,28,53
KIDNEY,40,60
LIHC,14,20
LUNG,29,47
PRAD,21,31
STAD,16,25


### now really make the pseudobulks

In [14]:
# double check we have the same regions and they are in the same order

np.all(tcga_adata.var.index == pbmc_adata.var.index)

True

In [15]:
tcga_adata.obs

,sample_id,scpred_CellType,isTraining,test_train_key
BRCA_000CFD9F_ADDF_4304_9E60_6041549E189C_X017_S06_L011_B1_T1_P040,BRCA_000CFD9F_ADDF_4304_9E60_6041549E189C_X017...,BRCA,Train,BRCA
BRCA_000CFD9F_ADDF_4304_9E60_6041549E189C_X017_S06_L012_B1_T2_P046,BRCA_000CFD9F_ADDF_4304_9E60_6041549E189C_X017...,BRCA,Train,BRCA
BRCA_01112370_4F6F_4A20_9BE0_7975C3465268_X017_S04_L007_B1_T1_P042,BRCA_01112370_4F6F_4A20_9BE0_7975C3465268_X017...,BRCA,Test,BRCA
BRCA_01112370_4F6F_4A20_9BE0_7975C3465268_X017_S04_L008_B1_T2_P044,BRCA_01112370_4F6F_4A20_9BE0_7975C3465268_X017...,BRCA,Train,BRCA
BRCA_0142AAAC_FFE8_43B7_AB99_02F7A1740567_X022_S06_L057_B1_T1_P050,BRCA_0142AAAC_FFE8_43B7_AB99_02F7A1740567_X022...,BRCA,Train,BRCA
...,...,...,...,...
STAD_D8BEB0DD_C346_4C1D_8A5F_6E669453AE6D_X027_S09_L094_B1_T2_PMRG,STAD_D8BEB0DD_C346_4C1D_8A5F_6E669453AE6D_X027...,STAD,Train,STAD
STAD_E07AAE91_7A4F_4FDC_9D67_4FA1D4E6DD89_X024_S05_L011_B1_T1_P061,STAD_E07AAE91_7A4F_4FDC_9D67_4FA1D4E6DD89_X024...,STAD,Train,STAD
STAD_E07AAE91_7A4F_4FDC_9D67_4FA1D4E6DD89_X024_S05_L012_B1_T2_P069,STAD_E07AAE91_7A4F_4FDC_9D67_4FA1D4E6DD89_X024...,STAD,Test,STAD
STAD_EA9A5DB6_348F_41C4_ACCC_5717653ABD74_X032_S06_L080_B1_T1_P080,STAD_EA9A5DB6_348F_41C4_ACCC_5717653ABD74_X032...,STAD,Test,STAD


In [16]:
# write out the gene ids
gene_pass = pbmc_adata.var['gene_ids']
gene_out_file = os.path.join(aug_data_path, f"{res_name}_genes.pkl")
gene_out_path = Path(gene_out_file)
pickle.dump( gene_pass, open( gene_out_path, "wb" ) )

# metadata

train_order = ['Train', 'Test']

# now generate all the proportions
total_meta_df = pd.DataFrame(columns = ["cancer_type", "num_cells", "isTraining"])

# no cell noise 
len_vector = pbmc_adata.obs["scpred_CellType"].unique().shape[0]
cell_noise = [np.random.lognormal(0, 0, pbmc_adata.var['gene_ids'].shape[0]) for i in range(len_vector)]

# cell type order
cell_order = pbmc_adata.obs.scpred_CellType.unique()

# simulate different number of cells
num_cells_vec = [100, 500, 1000, 5000, 5001]  # 5001 indicates that it should be ALL blood, using 5000 cells
num_cells_vec = [19000, 99000, 99900]  # 5% 1% .1%
idx = 0
for curr_train in train_order:
  pseudobulks_df_all = None
  prop_df_all = None
  metadata_df_all = None
  
  for curr_cells in num_cells_vec:

    # get the TCGA samples based on Test/Train
    curr_tcga_adata = tcga_adata[np.where(tcga_adata.obs.isTraining == curr_train)[0], :]

    # get the PBMC cells based on Test / Train
    curr_pbmc_adata = pbmc_adata[np.where(pbmc_adata.obs.isTraining == curr_train)[0], :]

    # we add all TCGA samples to the pseudobulks
    # but we will do this 10 times for replicates
    #tcga_adata_10 = curr_tcga_adata.concatenate(curr_tcga_adata,
    #                                  curr_tcga_adata)
    tcga_adata_10 = curr_tcga_adata


    print(f"running {curr_cells} {curr_train}")


    # make the pseudobulks

    print("make_prop_and_sum")
    prop_df, pseudobulks_df, _, _ = sc_preprocess.make_prop_and_sum(curr_pbmc_adata, 
                                                                    num_samples=tcga_adata_10.shape[0], 
                                                                    num_cells=curr_cells,
                                                                    use_true_prop=False,
                                                                    cell_noise=cell_noise,
                                                                    useSampleNoise=False)

    # add the TCGA sample to the pseudobulk
    # UNLESS its 5001, this means 5000 cells, but NO TCGA
    # this is our negative 
    if curr_cells != 5001:
      pseudobulks_df = pseudobulks_df + tcga_adata_10.X

    # make the metadata
    num_samps = pseudobulks_df.shape[0] 
    samp_type = ["bulk"]*num_samps
    
    metadata_df = pd.DataFrame(data = {"sample_id":tcga_adata_10.obs.sample_id, 
                                      "cancer_type":tcga_adata_10.obs.scpred_CellType,
                                      "isTraining":[curr_train]*num_samps,
                                      "num_blood":[curr_cells]*num_samps})

    # make the proportions instead of cell counts
    prop_df = prop_df.div(prop_df.sum(axis=1), axis=0)


    # put them together
    print("concat")    
    if pseudobulks_df_all is None:
      pseudobulks_df_all = pseudobulks_df
      prop_df_all = prop_df
      metadata_df_all = metadata_df
    else:
      prop_df_all = pd.concat([prop_df_all,prop_df])
      pseudobulks_df_all = pd.concat([pseudobulks_df_all, pseudobulks_df])
      metadata_df_all = pd.concat([metadata_df_all, metadata_df])


    pseudobulk_file = os.path.join(aug_data_path, f"{res_name}_{curr_cells}_STIM_{curr_train}_pseudo_splits.pkl")
    prop_file = os.path.join(aug_data_path, f"{res_name}_{curr_cells}_STIM_{curr_train}_prop_splits.pkl")
    meta_file = os.path.join(aug_data_path, f"{res_name}_{curr_cells}_STIM_{curr_train}_meta_splits.pkl")
    print("write")        
    pseudobulk_path = Path(pseudobulk_file)
    prop_path = Path(prop_file)
    meta_path = Path(meta_file)
    pickle.dump( prop_df, open( prop_path, "wb" ) )
    pickle.dump( pseudobulks_df, open( pseudobulk_path, "wb" ) )
    pickle.dump( metadata_df, open( meta_path, "wb" ) )





running 19000 Train
make_prop_and_sum


0
100
200
300
400
concat
write
running 99000 Train
make_prop_and_sum
0
100
200
300
400
concat
write
running 99900 Train
make_prop_and_sum
0
100
200
300
400
concat
write
running 19000 Test
make_prop_and_sum
0
100
200
concat
write
running 99000 Test
make_prop_and_sum
0
100
200
concat
write
running 99900 Test
make_prop_and_sum
0
100
200
concat
write
